<a href="https://colab.research.google.com/github/SomeiLam/ottomator-agents/blob/main/api-docs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install faiss-cpu openai tiktoken langchain crewai --upgrade --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 96.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 242.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 238.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 315.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.5/285.5 kB 331.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 291.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 278.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 215.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 243.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 154.3 MB/

In [2]:
import os, json, pathlib, textwrap, pickle, numpy as np, faiss, tiktoken, openai
from typing import List, Dict

In [3]:
import os, getpass, json, textwrap, pathlib, pickle, numpy as np, faiss, openai
from google.colab import userdata

api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

EMBED_MODEL = "text-embedding-3-small"

In [4]:
import json, pathlib, textwrap
FILE = pathlib.Path("sikka-apis.json")
root = json.loads(FILE.read_text())

def collect_docs(node):
    """
    Depth‑first walk over *any* Postman collection object (dict OR list) and
    return a list of {content:str, path:str}.
    """
    stack = [([], node)]
    out   = []

    while stack:
        path, cur = stack.pop()

        # ── handle dict nodes ───────────────────────────────────────────────
        if isinstance(cur, dict):
            name     = cur.get("name") or "<no‑name>"
            new_path = path + [name]

            buckets = []

            # 1) plain description
            if cur.get("description"):
                buckets.append(str(cur["description"]))

            # 2) request‑level docs + sample body
            req = cur.get("request", {})
            if isinstance(req, dict):
                if req.get("description"):
                    buckets.append(req["description"])
                raw = req.get("body", {}).get("raw")
                if raw: buckets.append("```json\n"+raw[:2000]+"\n```")

            # 3) response sample bodies
            for resp in cur.get("response", []):
                body = resp.get("body")
                if body:
                    buckets.append("```json\n"+body[:2000]+"\n```")

            # 4) event → script → exec (often holds markdown examples)
            for ev in cur.get("event", []):
                exec_lines = ev.get("script", {}).get("exec") or []
                if exec_lines:
                    buckets.append("\n".join(exec_lines)[:2000])

            # save doc
            if buckets:
                out.append({
                    "content": f"# {' › '.join(new_path)}\n\n" + "\n\n".join(buckets),
                    "path":    " › ".join(new_path)
                })

            # push children (if any)
            for child in cur.get("item", []):
                stack.append((new_path, child))

        # ── handle list nodes ───────────────────────────────────────────────
        elif isinstance(cur, list):
            for itm in cur:
                stack.append((path, itm))

    return out

docs = collect_docs(root)
print("Total docs captured:", len(docs))
print("\nFirst snippet ↓\n", docs[0]["content"][:300])


Total docs captured: 392

First snippet ↓
 # <no‑name>






In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter

# --- choose ONE of the two splitters -------------------------------------
# 1) Recursive (general‑purpose, keeps whole lines / paragraphs together first)
splitter = RecursiveCharacterTextSplitter(
    chunk_size      = 3000,     # target chars
    chunk_overlap   = 150,      # small overlap improves retrieval recall
    separators      = ["\n\n", "\n", " ", ""],  # try large -> small
)

# 2) Markdown‑aware (keeps headings and fenced code intact)
# splitter = MarkdownTextSplitter(chunk_size=3000, chunk_overlap=150)
# -------------------------------------------------------------------------

def smart_chunk_docs(docs):
    out = []
    for d in docs:
        pieces = splitter.split_text(d["content"])
        meta   = d.get("meta") or {"path": d.get("path", "")}
        for i, part in enumerate(pieces):
            out.append({"content": part, "meta": {**meta, "chunk": i}})
    return out


docs_raw   = collect_docs(root)        # 392 in your case
docs_chunk = smart_chunk_docs(docs_raw)
print("Docs before:", len(docs_raw))
print("Docs after :", len(docs_chunk))
print("Example meta:", docs_chunk[0]["meta"])


Docs before: 392
Docs after : 517
Example meta: {'path': '<no‑name>', 'chunk': 0}


In [6]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

def embed_texts(texts: list[str], model: str = "text-embedding-3-small"):
    """
    Returns a list[ list[float] ] of embeddings using the new OpenAI ≥1.0.0 SDK.
    """
    resp = client.embeddings.create(model=model, input=texts)
    # resp.data is a list of Embedding objects in **input order**
    return [e.embedding for e in resp.data]

vecs, BATCH = [], 96
for i in range(0, len(docs_raw), BATCH):
    batch = [d["content"] for d in docs_raw[i:i+BATCH]]
    vecs.extend(embed_texts(batch))

vecs = np.asarray(vecs, dtype="float32")
index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)
print("FAISS index size:", index.ntotal)

FAISS index size: 392


In [7]:
def ask_sikka(query, k=4):
    q_emb  = embed_texts([query])[0]         # returns 1×embedding
    _, ids = index.search(np.array([q_emb], dtype="float32"), k)
    context = "\n\n---\n\n".join(docs[i]["content"] for i in ids[0])

    chat = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system",
             "content":"Answer only from context; say 'Not found' otherwise."},
            {"role":"user",
             "content":f"Context:\n{context}\n\nQuestion: {query}"}
        ])
    return chat.choices[0].message.content.strip()

In [20]:
# Prompt template  ─────────────────────────────────────────────

OPTIMIZER_TEMPLATE = """
You are the Query Optimizer for a multi‑agent system that builds full‑stack apps with Sikka APIs.

USER_MESSAGE:
\"\"\"{user_message}\"\"\"

Step 1: Scope
• If the message is not about building or integrating Sikka APIs, output exactly:
  {{ "scope_ok": false, "reason": "short explanation" }}
  and STOP.

Step 2: Tech stack
• Detect any frontend frameworks/languages (e.g. React, Next.js, TypeScript).
• Detect any backend frameworks/languages (e.g. Node.js, Express, Python, Flask).
• If none are found, default to “React + JavaScript” front end and “Node.js + Express + JavaScript” back end.

Step 3: Rewrite & decompose
• Produce one sentence `optimized_query` restating the goal.
• Produce **four** prompts under `prompts` with these keys:

  1. **api_docs**
     – Identify all required Sikka endpoints for `optimized_query`.
     – For each endpoint, specify the **full base URL including version** (e.g. `https://api.sikkasoft.com/v4`), the path, and HTTP method.
     – Document the authentication flow (request_key lifecycle).
     – List required headers and body/query parameters per endpoint.
     – Include a sample request and a sample response for each endpoint.

  2. **frontend**
     – Build the UI using the detected frontend framework.
     – Call the endpoints identified in `api_docs`.
     – Specify component/file names.
     – Handle form state, loading indicators, and error displays.

  3. **backend**
     – Implement server routes using the detected backend framework.
     – Include code to obtain and refresh the request_key.
     – Show how to call each Sikka endpoint and return JSON responses.

  4. **formatter**
     – Assemble **all** outputs into a single Markdown document.
     – Include headings for Overview, API Documentation, Frontend Code, and Backend Code.

Step 4: Output exactly this JSON (no extra keys):

```json
{{
  "scope_ok": true,
  "optimized_query": "<one‑sentence restatement>",
  "tech_stack": {{
    "frontend": "<detected or default stack>",
    "backend":  "<detected or default stack>"
  }},
  "prompts": {{
    "api_docs":  "<instruction containing all required elements>",
    "frontend":  "<instruction containing all required elements>",
    "backend":   "<instruction containing all required elements>",
    "formatter": "<instruction containing all required elements>"
  }}
}}

"""


In [9]:
# CrewAi Agent
from crewai import Agent, Task, Crew
import re

from google.colab import userdata
client = OpenAI(api_key=api_key)

optimizer_agent = Agent(
  role = "Query Optimizer",
  goal = "Validate scope and normalise user queries about Sikka APIs",
  backstory = "Expert in Sikka’s API portfolio and software design.",
  allow_delegation = False,
  # verbose = False,
  llm = "gpt-4o-mini"
)

def run_optimizer(user_message: str):
  """Helper to execute the optimizer solo (for testing)."""
  prompt = OPTIMIZER_TEMPLATE.format(user_message=user_message)

  task = Task(
    agent = optimizer_agent,
    description = prompt,
    expected_output = "JSON exactly matching the schema above."
  )

  crew = Crew(agents=[optimizer_agent], tasks=[task], verbose=False)
  out    = crew.kickoff()                     # CrewOutput
  # 1) Grab the raw string from the first task
  raw = out.tasks_output[0].raw

  # 2) Strip any ```json fences at start/end
  #    This regex removes lines that are exactly ``` or ```json
  cleaned = "\n".join(
      line for line in raw.splitlines()
      if not re.match(r"^```(?:json)?\s*$", line)
  ).strip()

  # 3) Parse the clean JSON
  return json.loads(cleaned)

user_message = "I want a React checkout page that saves a patient's card and runs a $25 sale using Sikka sandbox."
print(run_optimizer(user_message))

{'scope_ok': True, 'optimized_query': "Create a React checkout page to save a patient's card and process a $25 transaction using Sikka sandbox.", 'tech_stack': {'frontend': 'React + JavaScript', 'backend': 'Node.js + Express + JavaScript'}, 'prompts': {'api_docs': "Identify all required Sikka endpoints for creating a React checkout page that saves a patient's card and processes a $25 sale. Document the authentication flow including the request_key lifecycle, list required headers and body/query parameters for each endpoint. Include a sample request and sample response for every endpoint.", 'frontend': 'Build the UI using React. Include routes/endpoints to call as specified in the api_docs. Specify component/file names to manage form state, handle loading, and errors.', 'backend': 'Implement the server routes using Node.js and Express. Include code to obtain and refresh the request_key. Show the implementation for each Sikka endpoint and ensure that responses are returned in JSON format

In [10]:
# Define the API‑Docs agent ────────────────────────────────────────
from crewai import Agent, Task, Crew

# Helper: FAISS retriever ─────────────────────────────────────────
def retrieve_api_context(query: str, k: int = 6) -> str:
    """Return the top‑k passages from your FAISS index for this query."""
    q_emb  = embed_texts([query])[0]
    D, idx = index.search(np.array([q_emb],dtype="float32"), k)
    return "\n\n---\n\n".join(docs[i]["content"] for i in idx[0])

api_doc_agent = Agent(
    role = "API Docs Generator",
    goal = "Produce detailed API documentation for the endpoints needed",
    backstory = "Specialist in Sikka API reference docs.",
    allow_delegation = False,
    llm = "gpt-4o-mini"
)

# Connect optimizer → api‑docs with Tasks ────────────────────────

# ── 3. Mini‑pipeline: API‑Docs step ────────────────────────────────────
def run_api_docs_step(opt_out: dict) -> str:
    if not opt_out.get("scope_ok", False):
        return "⚠️ Out of scope. No API docs generated."

    # 1) Build the literal prompt (only the api_docs instruction + FAISS context)
    instr  = opt_out["prompts"]["api_docs"]
    ctx    = retrieve_api_context(opt_out["optimized_query"])
    prompt = f"{instr}\n\nHere is the relevant API context:\n{ctx}"

    # 2) Single‑task Crew
    task = Task(
        agent           = api_doc_agent,
        description     = prompt,
        expected_output = (
            "A Markdown document covering ONLY the Sikka endpoints, "
            "authentication flow, headers, parameters, and sample calls."
        )
    )
    crew_output = Crew(
        agents=[api_doc_agent],
        tasks=[task],
        verbose=False
    ).kickoff()

    # 3) Extract the raw Markdown string from that one task
    #    (TaskOutput.raw holds the LLM's literal response)
    api_md = crew_output.tasks_output[0].raw

    return api_md


In [11]:
# ── 1) Define the Frontend Code Generator agent ─────────────────────────
from crewai import Agent, Task, Crew

frontend_agent = Agent(
    role             = "Frontend Code Generator",
    goal             = "Produce React/Next.js frontend code for the checkout flow",
    backstory        = "Expert in building modern React applications and UI/UX best practices.",
    allow_delegation = False,
    llm              = "gpt-4o-mini"
)

# ── 2) Frontend runner helper ──────────────────────────────────────────
def run_frontend_step(opt_out: dict, api_docs_md: str) -> str:
    """
    Generate the React/Next.js frontend code.

    Parameters:
    - opt_out: dict from run_optimizer(...)
    - api_docs_md: Markdown string from run_api_docs_step(...)

    Returns:
    - Raw code string for the frontend
    """
    # 1) Out‑of‑scope guard
    if not opt_out.get("scope_ok", False):
        return "⚠️ Out of scope. No frontend code generated."

    # 2) Build the literal prompt
    instr  = opt_out["prompts"]["frontend"]
    prompt = (
        f"{instr}\n\nHere are the API docs you should integrate with:\n"
        f"{api_docs_md}"
    )

    # 3) Single‑task Crew for frontend code
    task = Task(
        agent           = frontend_agent,
        description     = prompt,
        expected_output = (
            "A React (or Next.js) component/file structure & code for the checkout UI, "
            "handling card input, form state, loading, and error display."
        )
    )
    crew_output = Crew(agents=[frontend_agent], tasks=[task], verbose=False).kickoff()

    # 4) Return only the raw code
    return crew_output.tasks_output[0].raw


In [12]:
# ── 1) Define the Backend Code Generator agent ───────────────────────────
from crewai import Agent, Task, Crew

backend_agent = Agent(
    role             = "Backend Code Generator",
    goal             = "Produce Node.js/Express backend code for the checkout flow",
    backstory        = "Expert in building secure and scalable Express APIs.",
    allow_delegation = False,
    llm              = "gpt-4o-mini"
)

# ── 2) Backend runner helper ────────────────────────────────────────────
def run_backend_step(opt_out: dict, api_docs_md: str) -> str:
    """
    Generate the Node.js + Express backend code.

    Parameters:
    - opt_out: dict from run_optimizer(...)
    - api_docs_md: Markdown string from run_api_docs_step(...)

    Returns:
    - Raw code string for the backend
    """
    # 1) Out‑of‑scope guard
    if not opt_out.get("scope_ok", False):
        return "⚠️ Out of scope. No backend code generated."

    # 2) Build the literal prompt
    instr  = opt_out["prompts"]["backend"]
    prompt = (
        f"{instr}\n\nHere are the API docs you should integrate with:\n"
        f"{api_docs_md}"
    )

    # 3) Single‑task Crew for backend code
    task = Task(
        agent           = backend_agent,
        description     = prompt,
        expected_output = (
            "Node.js + Express server code with routes to handle saving cards "
            "and processing payments using the provided Sikka endpoints."
        )
    )
    crew_output = Crew(agents=[backend_agent], tasks=[task], verbose=False).kickoff()

    # 4) Return only the raw code
    return crew_output.tasks_output[0].raw

In [13]:
import json
import re
from crewai import Agent, Task, Crew

# ── 1) Define the Frontend Evaluator agent ─────────────────────────────
frontend_evaluator_agent = Agent(
    role             = "Frontend Code Evaluator",
    goal             = (
        "Inspect the React/Next.js frontend code and report *only* critical "
        "correctness or security issues in JSON format."
    ),
    backstory        = "Expert in React best practices, form validation, and front‑end security.",
    allow_delegation = False,
    llm              = "gpt-4o-mini"
)

# ── 2) Runner helper for frontend evaluation ─────────────────────────────
def run_frontend_eval(opt_out: dict, frontend_code: str) -> dict:
    """
    Analyze frontend code for blocking issues.
    Returns a dict with:
      {
        "issues": [
          {"location":"frontend","line":<int>,"message":"<desc>","severity":"critical"},
          ...
        ],
        "suggestions": [
          "<actionable improvement>",
          ...
        ]
      }
    """
    # Out‑of‑scope guard
    if not opt_out.get("scope_ok", False):
        return {"issues": [], "suggestions": []}

    prompt = f"""
You are a strict Bug‑Finder focused ONLY on frontend code.
Report *only* critical correctness or security issues in this JSON schema:

{{
  "issues": [
    {{
      "location": "frontend",
      "line": <integer>,
      "message": "<description>",
      "severity": "critical"
    }},
    ...
  ],
  "suggestions": [
    "<actionable improvement>",
    ...
  ]
}}

Here is the frontend code to evaluate:
{frontend_code}

Do NOT include any markdown or additional keys—output only the JSON.
""".strip()

    task = Task(
        agent           = frontend_evaluator_agent,
        description     = prompt,
        expected_output = "A JSON object with keys 'issues' and 'suggestions' as specified."
    )
    out = Crew(agents=[frontend_evaluator_agent], tasks=[task], verbose=False).kickoff()
    raw = out.tasks_output[0].raw

    # Strip any ``` fences
    cleaned = "\n".join(
        line for line in raw.splitlines()
        if not re.match(r"^```(?:\w+)?\s*$", line)
    ).strip()

    return json.loads(cleaned)


In [14]:
import json
import re
from crewai import Agent, Task, Crew

# ── 1) Define the Backend Evaluator agent ──────────────────────────────
backend_evaluator_agent = Agent(
    role             = "Backend Code Evaluator",
    goal             = (
        "Inspect the Node.js/Express backend code and report *only* critical "
        "correctness or security issues in JSON format."
    ),
    backstory        = "Expert in Express API design, authentication flows, and backend security.",
    allow_delegation = False,
    llm              = "gpt-4o-mini"
)

# ── 2) Runner helper for backend evaluation ─────────────────────────────
def run_backend_eval(opt_out: dict, backend_code: str) -> dict:
    """
    Analyze backend code for blocking issues.
    Returns a dict matching:
      {
        "issues": [
          {"location":"backend","line":<int>,"message":"<desc>","severity":"critical"},
          ...
        ],
        "suggestions": [
          "<actionable improvement>",
          ...
        ]
      }
    """
    # Out‑of‑scope guard
    if not opt_out.get("scope_ok", False):
        return {"issues": [], "suggestions": []}

    prompt = f"""
You are a strict Bug‑Finder focused ONLY on backend code.
Report *only* critical correctness or security issues in this JSON schema:

{{
  "issues": [
    {{
      "location": "backend",
      "line": <integer>,
      "message": "<description>",
      "severity": "critical"
    }},
    ...
  ],
  "suggestions": [
    "<actionable improvement>",
    ...
  ]
}}

Here is the backend code to evaluate:
{backend_code}

Do NOT include any markdown or additional keys—output only the JSON.
""".strip()

    task = Task(
        agent           = backend_evaluator_agent,
        description     = prompt,
        expected_output = "A JSON object with keys 'issues' and 'suggestions' as specified."
    )
    out = Crew(agents=[backend_evaluator_agent], tasks=[task], verbose=False).kickoff()
    raw = out.tasks_output[0].raw

    # Strip any ``` fences if present
    cleaned = "\n".join(
        line for line in raw.splitlines()
        if not re.match(r"^```(?:\w+)?\s*$", line)
    ).strip()

    return json.loads(cleaned)


In [15]:
import json, re
from crewai import Agent, Task, Crew

# Define a reusable Code Refinement Agent
refine_agent = Agent(
    role             = "Code Refinement Agent",
    goal             = (
        "Incorporate evaluator feedback into an existing code snippet,\
        and return the revised code only."
    ),
    backstory        = "Expert in iterative code improvement and best practices.",
    allow_delegation = False,
    llm              = "gpt-4o-mini"
)

# Runner helper for code refinement
def run_refine_step(
    opt_out: dict,            # Optimizer output dict
    code_snippet: str,       # Original code (frontend or backend)
    eval_report: dict,       # Evaluation report with issues & suggestions
    code_type: str           # "frontend" or "backend"
) -> str:
    """
    Refines the given code snippet by applying evaluator feedback.

    Returns the refined code as a raw string (no additional text).
    """
    # Out-of-scope guard
    if not opt_out.get("scope_ok", False):
        return code_snippet

    # Prepare feedback list
    feedback_lines = [f"- {s}" for s in eval_report.get("suggestions", [])]
    feedback_text  = "\n".join(feedback_lines)

    # Build prompt
    prompt = f"""
You are a Code Refinement Agent. Update the following {code_type} code using the evaluator feedback.

Evaluator Report:
{json.dumps(eval_report, indent=2)}

Original {code_type.capitalize()} Code:
```
{code_snippet}
```

Please return **only** the refined {code_type} code, applying all critical fixes. Do not include any explanations.
""".strip()

    # Create and run the refinement task
    task = Task(
        agent           = refine_agent,
        description     = prompt,
        expected_output = (
            f"The updated {code_type} code snippet, with fixes applied."
        )
    )
    crew_output = Crew(agents=[refine_agent], tasks=[task], verbose=False).kickoff()

    # Extract the raw code response
    raw = crew_output.tasks_output[0].raw
    # Strip code fences if present
    cleaned = "\n".join(
        line for line in raw.splitlines()
        if not re.match(r"^```(?:\w+)?\s*$$", line)
    )
    return cleaned


In [16]:
from concurrent.futures import ThreadPoolExecutor

def _pipeline_frontend(opt_out: dict, api_docs_md: str) -> str:
    """
    Full frontend pipeline: generate → evaluate → refine
    Returns the final refined frontend code.
    """
    # 1) Generate initial code
    code = run_frontend_step(opt_out, api_docs_md)

    # 2) Evaluate it
    eval_report = run_frontend_eval(opt_out, code)
    print("[DEBUG] Frontend eval_report:", eval_report, flush=True)

    # 3) If there are critical issues, refine
    if eval_report["issues"]:
        code = run_refine_step(opt_out, code, eval_report, code_type="frontend")

    return code

def _pipeline_backend(opt_out: dict, api_docs_md: str) -> str:
    """
    Full backend pipeline: generate → evaluate → refine
    Returns the final refined backend code.
    """
    # 1) Generate initial code
    code = run_backend_step(opt_out, api_docs_md)

    # 2) Evaluate it
    eval_report = run_backend_eval(opt_out, code)
    print("[DEBUG] Backend eval_report:", eval_report, flush=True)

    # 3) If there are critical issues, refine
    if eval_report["issues"]:
        code = run_refine_step(opt_out, code, eval_report, code_type="backend")

    return code

def run_frontend_and_backend(opt_out: dict, api_docs_md: str) -> tuple[str, str]:
    """
    Runs both the frontend and backend full pipelines in parallel.
    Returns a tuple (final_frontend_code, final_backend_code).
    """
    with ThreadPoolExecutor(max_workers=2) as executor:
        fut_frontend = executor.submit(_pipeline_frontend, opt_out, api_docs_md)
        fut_backend  = executor.submit(_pipeline_backend,  opt_out, api_docs_md)

        final_frontend = fut_frontend.result()
        final_backend  = fut_backend.result()

    return final_frontend, final_backend


In [18]:
import re
from crewai import Agent, Task, Crew

# ── 1) Define the Formatter Agent ─────────────────────────────────────────
formatter_agent = Agent(
    role             = "Documentation Formatter",
    goal             = (
        "Assemble the optimized query, tech stack, API docs, frontend code, "
        "and backend code into one coherent Markdown document."
    ),
    backstory        = "Detail‑oriented technical writer and developer.",
    allow_delegation = False,
    llm              = "gpt-4o-mini"
)

# ── 2) Runner helper for the formatter ────────────────────────────────────
def run_formatter_step(
    opt_out: dict,
    api_docs_md: str,
    frontend_code: str,
    backend_code: str
) -> str:
    """
    Takes the optimizer output dict plus the generated API docs, frontend code,
    and backend code, and returns a single Markdown document.
    """
    # Guard: only format if in‑scope
    if not opt_out.get("scope_ok", False):
        return "⚠️ Out of scope. Nothing to format."

    # Build the literal prompt
    prompt = f"""
You are the Documentation Formatter.  Using the pieces below, produce **only**
a Markdown document with these top‑level headings in this order:

# Overview
# API Documentation
# Frontend Code
# Backend Code

Include the optimized query and tech stack under Overview, insert the raw API docs
under API Documentation, then wrap the frontend and backend code each in fenced
```javascript blocks.

### Overview

**Task:** {opt_out['optimized_query']}

**Tech Stack:**
- Frontend: {opt_out['tech_stack']['frontend']}
- Backend: {opt_out['tech_stack']['backend']}

### API Documentation

{api_docs_md}

### Frontend Code

```javascript
{frontend_code}
```

### Backend Code
```
{backend_code}
```

Return only the Markdown—no extra commentary. """.strip()

    # Create and run the formatting task
    task = Task(
        agent           = formatter_agent,
        description     = prompt,
        expected_output = "A single Markdown document with the specified sections."
    )
    crew_output = Crew(
        agents=[formatter_agent],
        tasks=[task],
        verbose=False
    ).kickoff()

    # Extract the raw Markdown
    md = crew_output.tasks_output[0].raw

    # Strip any accidental code fences around the entire doc
    if md.startswith("```"):
        md = "\n".join(line for line in md.splitlines()
                      if not re.match(r"^```", line)).strip()

    return md

In [19]:
def orchestrate(user_message: str) -> str:
    """
    Runs the full Sikka checkout pipeline:
      1) Optimize the user query
      2) Generate API docs
      3) Generate & auto‑fix frontend + backend code
      4) Format everything into Markdown

    Returns the final Markdown (or an out‑of‑scope notice).
    """
    # 1) Optimize
    opt_out = run_optimizer(user_message)
    print("🔍 Optimizer Output:", opt_out, "\n" + "─" * 60, flush=True)
    if not opt_out.get("scope_ok", False):
        return f"⚠️ Out of scope: {opt_out.get('reason','')}"

    # 2) API docs
    api_docs = run_api_docs_step(opt_out)
    print("📄 API Documentation (snippet):", api_docs[:300], "\n" + "─" * 60, flush=True)

    # 3) Generate & refine code
    frontend_code, backend_code = run_frontend_and_backend(opt_out, api_docs)
    print("🚀 Frontend Code (snippet):", frontend_code[:200], flush=True)
    print("🔧 Backend  Code (snippet):", backend_code[:200], "\n" + "─" * 60, flush=True)

    # 4) Format into Markdown
    final_md = run_formatter_step(opt_out, api_docs, frontend_code, backend_code)
    print("🎉 Final Markdown generated.", flush=True)

    return final_md


# ── Example usage ───────────────────────────────────────────
if __name__ == "__main__":
    message = (
        "I want a React checkout page that saves a patient's card "
        "and runs a $25 sale using Sikka sandbox."
    )
    doc = orchestrate(message)
    print("\n=== Final Output ===\n")
    print(doc)
